In [4]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error , plot_confusion_matrix
import matplotlib.pyplot as plt
import datetime as dt
import time
from math import sqrt

In [5]:
data = pd.read_csv('supplement.csv')
data

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,Yes,51,37272.00
188336,T1188337,153,S4,L2,R1,2019-05-31,1,No,90,54572.64
188337,T1188338,154,S1,L3,R2,2019-05-31,1,No,56,31624.56
188338,T1188339,155,S3,L1,R2,2019-05-31,1,Yes,70,49162.41


In [11]:
data.shape

(188340, 10)

### our dataset consists of 188340 observations and 10 attributes

In [6]:
data.columns

Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Date',
       'Holiday', 'Discount', '#Order', 'Sales'],
      dtype='object')

In [7]:
data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [8]:
data.tail()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
188335,T1188336,149,S2,L3,R2,2019-05-31,1,Yes,51,37272.00
188336,T1188337,153,S4,L2,R1,2019-05-31,1,No,90,54572.64
188337,T1188338,154,S1,L3,R2,2019-05-31,1,No,56,31624.56
188338,T1188339,155,S3,L1,R2,2019-05-31,1,Yes,70,49162.41
188339,T1188340,152,S2,L1,R1,2019-05-31,1,No,47,37977.00


In [9]:
data.info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


our dataset consists of 6 categorical and 4 numeric variables, no missing values

In [14]:
# describe
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Store_id,188340.0,183.000000,105.366308,1.0,92.0,183.0,274.0,365.0
Holiday,188340.0,0.131783,0.338256,0.0,0.0,0.0,0.0,1.0
#Order,188340.0,68.205692,30.467415,0.0,48.0,63.0,82.0,371.0
Sales,188340.0,42784.327982,18456.708302,0.0,30426.0,39678.0,51909.0,247215.0


In [25]:
# unique value counts
data.nunique()

Store_id           365
Store_Type           4
Location_Type        5
Region_Code          4
Date               516
Holiday              2
Discount             2
#Order             299
Sales            47422
dtype: int64

In [18]:
# remove the id arm;
data.drop("ID",axis=1,inplace=True)
data.head()

,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [19]:
col=["Store_Type","Location_Type","Region_Code","Holiday","Discount"]

for i in col:
    print(data[i].value_counts())
    print("*"*30)

S1    88752
S4    45924
S2    28896
S3    24768
Name: Store_Type, dtype: int64
******************************
L1    85140
L2    48504
L3    29928
L5    13932
L4    10836
Name: Location_Type, dtype: int64
******************************
R1    63984
R2    54180
R3    44376
R4    25800
Name: Region_Code, dtype: int64
******************************
0    163520
1     24820
Name: Holiday, dtype: int64
******************************
No     104051
Yes     84289
Name: Discount, dtype: int64
******************************


In [26]:
from datetime import datetime 

In [28]:
data["Date"] = pd.to_datetime(data.Date)

In [33]:
data['year'] = data['Date'].dt.year 
data['month'] = data['Date'].dt.month 
data["day"]=data["Date"].dt.day
data.head()

,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales,year,month,day
0,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84,2018,1,1
1,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12,2018,1,1
2,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20,2018,1,1
3,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16,2018,1,1
4,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52,2018,1,1


In [35]:
# list for numeric and categorical values;

cat_list=[]
num_list=[]

for i in data.columns:
    unique_val=len(data[i].unique())
    
    if unique_val<32:
        cat_list.append(i)
    else:
        num_list.append(i)

In [36]:
cat_list

['Store_Type',
 'Location_Type',
 'Region_Code',
 'Holiday',
 'Discount',
 'year',
 'month',
 'day']

In [37]:
num_list

['Store_id', 'Date', '#Order', 'Sales']